In [1]:
!git clone https://github.com/parkerburchett/Numerai

Cloning into 'Numerai'...
remote: Enumerating objects: 1522, done.
remote: Counting objects: 100% (1522/1522), done.
remote: Compressing objects: 100% (1341/1341), done.
remote: Total 1522 (delta 241), reused 1434 (delta 161), pack-reused 0
Receiving objects: 100% (1522/1522), 76.08 MiB | 26.81 MiB/s, done.
Resolving deltas: 100% (241/241), done.


In [2]:
from Numerai.SimpleScoreCalculator import NumeraiDataLoader
# this works.
help(NumeraiDataLoader)

Help on class NumeraiDataLoader in module Numerai.SimpleScoreCalculator:

class NumeraiDataLoader(builtins.object)
 |  Pings and cleans the raw data from Numerai. Each method returns a DataFrame that is optimized for saving memory space.
 |  
 |  Much of the code is copy pasted. You can replace fix that later.
 |  
 |  Methods defined here:
 |  
 |  ping_example_predictions(self) -> pandas.core.frame.DataFrame
 |      Create a dataframe of id, Prediction, rank_prediction. Run time is ~2 second
 |      
 |      
 |      id              : The unique identifier for a row in the tournament data provided by Numerai
 |      prediction      : A float (0,1) that the example model predicts for that row. 
 |      rank_prediction : 'prediction' after it is rank normalized.
 |      
 |      
 |      
 |      Example : 
 |      
 |                                prediction  rank_prediction
 |      id              
 |      n0003aa52cab36c2        0.49    0.097334
 |      n000920ed083903f        0.49

In [3]:
dl = NumeraiDataLoader()

In [4]:
%%time
example_preds = dl.ping_example_predictions()
print(example_preds.shape)
print(example_preds.head())

(1736405, 2)
                  prediction  rank_example_prediction
id                                                   
n0003aa52cab36c2        0.49                 0.097364
n000920ed083903f        0.49                 0.097364
n0038e640522c4a6        0.53                 0.969351
n004ac94a87dc54b        0.51                 0.656619
n0052fe97ea0c05f        0.50                 0.332463
CPU times: user 3.25 s, sys: 279 ms, total: 3.53 s
Wall time: 4.87 s


In [5]:
len(example_preds['rank_example_prediction'].unique()) # looks good each id has a unique prediction

1736405

In [6]:
%%time
valid_df = dl.ping_validation_data()
print(valid_df.shape)
print(valid_df.head())

(137779, 313)
   era  feature_intelligence1  ...  target  rank_target
0  121                      1  ...    0.25     0.049935
1  121                      3  ...    0.50     0.249755
2  121                      4  ...    1.00     0.950058
3  121                      3  ...    0.50     0.249762
4  121                      1  ...    0.75     0.750223

[5 rows x 313 columns]
CPU times: user 1min 21s, sys: 12.6 s, total: 1min 34s
Wall time: 1min 39s


In [17]:
valid_df['era'].value_counts()

210    5227
211    5197
212    5191
209    5164
205    5161
204    5152
206    5143
203    5119
208    5114
202    5090
201    5083
199    5006
207    4991
198    4981
197    4970
200    4929
131    4814
132    4812
130    4756
129    4705
125    4698
127    4688
126    4682
122    4658
128    4636
124    4630
123    4609
121    4573
Name: era, dtype: int64

In [7]:
%%time
tournament_df = dl.ping_tournament_data() 
print(tournament_df.shape)
print(tournament_df.head())

Traceback (most recent call last):
  File "/content/Numerai/SimpleScoreCalculator.py", line 363, in ping_tournament_data
    valid_df["era"] = valid_df["era"].apply(lambda x: int(x[3:])) # strip the word 'era' from the era column and cast as an int
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/series.py", line 4213, in apply
    mapped = lib.map_infer(values, f, convert=convert_dtype)
  File "pandas/_libs/lib.pyx", line 2403, in pandas._libs.lib.map_infer
  File "/content/Numerai/SimpleScoreCalculator.py", line 363, in <lambda>
    valid_df["era"] = valid_df["era"].apply(lambda x: int(x[3:])) # strip the word 'era' from the era column and cast as an int
ValueError: invalid literal for int() with base 10: 'X'


(1736405, 311)
                     era  ...  feature_wisdom46
id                        ...                  
n0003aa52cab36c2  era121  ...                 0
n000920ed083903f  era121  ...                 2
n0038e640522c4a6  era121  ...                 0
n004ac94a87dc54b  era121  ...                 1
n0052fe97ea0c05f  era121  ...                 4

[5 rows x 311 columns]
CPU times: user 4min 46s, sys: 27.8 s, total: 5min 13s
Wall time: 5min 17s


In [8]:
%%time
train_df = dl.ping_training_data()
print(train_df.shape)
print(train_df.head())

(501808, 314)
                  era data_type  ...  target  rank_target
id                               ...                     
n000315175b67977    1     train  ...    0.50     0.249239
n0014af834a96cdd    1     train  ...    0.25     0.049854
n001c93979ac41d4    1     train  ...    0.25     0.049856
n0034e4143f22a13    1     train  ...    0.25     0.049858
n00679d1a636062f    1     train  ...    0.75     0.750779

[5 rows x 314 columns]
CPU times: user 1min 22s, sys: 12.2 s, total: 1min 34s
Wall time: 1min 36s


### See how much memory each dataframe uses

In [9]:
print('Example Prediction DataFrame')
print(example_preds.info(memory_usage="deep"))

Example Prediction DataFrame
<class 'pandas.core.frame.DataFrame'>
Index: 1736405 entries, n0003aa52cab36c2 to nffe50a4328f828a
Data columns (total 2 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   prediction               float64
 1   rank_example_prediction  float64
dtypes: float64(2)
memory usage: 147.4 MB
None


In [10]:
print('Tournament_data DataFrame')
print(tournament_df.info(memory_usage="deep"))

Tournament_data DataFrame
<class 'pandas.core.frame.DataFrame'>
Index: 1736405 entries, n0003aa52cab36c2 to nffe50a4328f828a
Columns: 311 entries, era to feature_wisdom46
dtypes: object(1), uint8(310)
memory usage: 738.6 MB
None


In [11]:
print('Validation Data DataFrame')
print(valid_df.info(memory_usage="deep"))

Validation Data DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137779 entries, 0 to 137778
Columns: 313 entries, era to rank_target
dtypes: float64(2), int64(1), uint8(310)
memory usage: 43.9 MB
None


In [12]:
print('Train DataFrame')
print(train_df.info(memory_usage="deep"))

Train DataFrame
<class 'pandas.core.frame.DataFrame'>
Index: 501808 entries, n000315175b67977 to nffba5460b572cfa
Columns: 314 entries, era to rank_target
dtypes: float64(2), int64(1), object(1), uint8(310)
memory usage: 224.4 MB
None


### Total memory usage

1.1 GB.

If you just want the training Data:
.22 GB.

If you want train and valid:
.26 GB

If you want train and valid and tournament:
1GB